# LM317 Voltage Regulator Datasheet Structured Extraction

<a href="https://colab.research.google.com/github/run-llama/llama_cloud_services/blob/main/examples/extract/lm317_structured_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates an agentic document workflow using LlamaExtract to process an LM317 voltage regulator datasheet. In this example, we define a structured extraction schema that converts key technical fields into standardized subfields. For instance, the output voltage is split into a minimum and maximum value with a defined unit, and we capture page citations for each extracted field.

The target user is an electronics engineer at a component manufacturing company who needs to consolidate datasheet information into a standardized specification sheet for design and quality control.

This approach reduces manual data entry, improves extraction accuracy and standardization, and provides traceability for each technical detail.

## Use Case Overview

### Problem
Datasheets like that for the LM317 regulator are often distributed as PDFs containing multiple tables, charts, and complex textual descriptions. Engineers must manually extract technical details such as voltage ranges, dropout voltage, maximum current, input voltage range, and pin configurations. This process is error-prone and time-consuming.

### Agent Workflow (Combination of Automation and Chat)
1. **Upload Datasheet:** The engineer uploads the LM317 datasheet PDF.  
2. **Structured Extraction:** An automated agent processes the PDF and extracts key technical details into structured fields (e.g., output voltage as a range with separate min/max values).
3. **Interactive Verification:** The engineer can query the agent (via chat) for further details or clarification (e.g., "Show me the detailed pin configuration extraction") and review the cited pages.

**Value Delivered:**
- Up to 70% reduction in manual data extraction time.
- Increased accuracy and standardization with structured fields.

## Setup and Download Data

Download the [LM317 Datasheet](https://www.ti.com/lit/ds/symlink/lm317.pdf) and setup LlamaExtract.

In [ ]:
!mkdir -p data/lm317_structured_extraction
!wget https://www.ti.com/lit/ds/symlink/lm317.pdf -O data/lm317_structured_extraction/lm317.pdf

In [ ]:
from dotenv import load_dotenv
from llama_cloud_services import LlamaExtract
from llama_cloud.core.api_error import ApiError

# Load environment variables (ensure LLAMA_CLOUD_API_KEY is set in your .env file)
load_dotenv(override=True)

# Initialize the LlamaExtract client
llama_extract = LlamaExtract(
    project_id="<project_id>",
    organization_id="<organization_id>",
)

## Defining a Structured Extraction Schema

We now define a rich Pydantic schema to extract technical specifications from the LM317 datasheet. In this schema:

- The **output_voltage** and **input_voltage** fields are structured as ranges with separate minimum and maximum values and a unit.
- The **pin_configuration** field is structured to include a pin count and a descriptive layout.
- Additional technical fields (e.g., dropout voltage, max current) are captured as numbers.


In [ ]:
from pydantic import BaseModel, Field
from typing import List


class VoltageRange(BaseModel):
    min_voltage: float = Field(..., description="Minimum voltage in volts")
    max_voltage: float = Field(..., description="Maximum voltage in volts")
    unit: str = Field("V", description="Voltage unit")


class PinConfiguration(BaseModel):
    pin_count: int = Field(..., description="Number of pins")
    layout: str = Field(..., description="Detailed pin layout description")


class LM317Spec(BaseModel):
    component_name: str = Field(..., description="Name of the component")
    output_voltage: VoltageRange = Field(
        ..., description="Output voltage range specification"
    )
    dropout_voltage: float = Field(..., description="Dropout voltage in volts")
    max_current: float = Field(..., description="Maximum current rating in amperes")
    input_voltage: VoltageRange = Field(
        ..., description="Input voltage range specification"
    )
    pin_configuration: PinConfiguration = Field(
        ..., description="Pin configuration details"
    )
    features: List[str] = Field([], description="List of additional technical features")


class LM317Schema(BaseModel):
    specs: List[LM317Spec] = Field(
        ..., description="List of extracted LM317 technical specifications"
    )

In [ ]:
try:
    existing_agent = llama_extract.get_agent(name="lm317-datasheet")
    if existing_agent:
        llama_extract.delete_agent(existing_agent.id)
except ApiError as e:
    if e.status_code == 404:
        pass
    else:
        raise

Here we use our balanced extraction mode.

In [ ]:
from llama_cloud import ExtractConfig

extract_config = ExtractConfig(
    extraction_mode="BALANCED",
)

agent = llama_extract.create_agent(
    name="lm317-datasheet", data_schema=LM317Schema, config=extract_config
)

## Extracting Information from the LM317 Datasheet

For this demonstration, please download a publicly available LM317 voltage regulator datasheet (for example, from Texas Instruments) and save it as `lm317.pdf` in the `./data` directory. Then run the cell below to extract the structured technical specifications.

In [ ]:
# Path to the LM317 datasheet PDF
lm317_pdf = "./data/lm317_structured_extraction/lm317.pdf"

# Extract structured technical specifications from the datasheet
lm317_extract = agent.extract(lm317_pdf)

Extracting files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:27<00:00, 87.38s/it]


## Assessing the Extraction Results

The output will be a consolidated list of LM317 technical specifications. For each entry, you should see structured fields including:

- **component_name**
- **output_voltage** as a range (with separate `min_voltage` and `max_voltage` plus `unit`)
- **dropout_voltage** and **max_current** as numbers
- **input_voltage** as a structured range
- **pin_configuration** with a `pin_count` and `layout`
- **features** (if available)

This structured approach makes it easier to standardize the information for downstream integration and verification. Engineers can click on the cited page numbers (in a UI that supports it) to validate the extraction.

In [ ]:
# Display the extraction results
lm317_extract.data

{'specs': [{'component_name': 'LM317',
   'output_voltage': {'min_voltage': 1.25, 'max_voltage': 37.0, 'unit': 'V'},
   'dropout_voltage': 0.0,
   'max_current': 1.5,
   'input_voltage': {'min_voltage': 4.25, 'max_voltage': 40.0, 'unit': 'V'},
   'pin_configuration': {'pin_count': 3,
    'layout': '1: ADJUST, 2: OUTPUT, 3: INPUT'},
   'features': ['Output voltage range adjustable from 1.25 V to 37 V',
    'Output current greater than 1.5 A',
    'Internal short-circuit current limiting',
    'Thermal overload protection',
    'Output safe-area compensation']}]}

You can also see the output result in the UI.

![](data/lm317_structured_extraction/lm317_extraction.png)

## Conclusion

This notebook demonstrated how to use LlamaExtract with a structured extraction schema for the LM317 voltage regulator datasheet. By defining detailed subfields (such as splitting voltage ranges into minimum and maximum values, and structuring the pin configuration), we ensure that the extracted data is standardized and traceable through page citations. This approach minimizes manual effort and improves accuracy, providing a robust example of an agentic document workflow for technical documentation processing.

Feel free to modify or extend the schema to capture additional technical details or to suit your own use cases.